In [5]:
import time
import logging
import numpy as np
import pandas as pd
import seaborn as sns
from time import time
from pathlib import Path
from pprint import pprint
from pandas import read_csv
from datetime import datetime
import matplotlib.pyplot as plt

# File system manangement
import os
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# Sklearn processing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler, OneHotEncoder
from sklearn.model_selection import  StratifiedShuffleSplit,ShuffleSplit,RepeatedStratifiedKFold,StratifiedKFold

# Sklearn regression algorithms regression models.
from time import time
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression,SGDRegressor,Ridge,LinearRegression,Lasso
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor

# Sklearn regression algorithms
# Sklearn regression model evaluation functions
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score

In [7]:
path = "/Users/surelmanda/Downloads/AirGUARD/Gas_Turbine_Streaming/gas_turbine_full.csv"
# load all data from file that we will use for your homework
data = pd.read_csv(path,sep=',')


print("Shape of the original dataset: ")
print('data_Train.shape :' ,data.shape)

# resetting index
data.reset_index(inplace = True,drop=True)
# sort by index labels
data.sort_index(axis = 0,inplace = True) 

# Renommer les colonnes spécifiées
data = data.rename(columns={'Unnamed: 0': 'ID'})
display(data.head())

Shape of the original dataset: 
data_Train.shape : (36733, 12)


,ID,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,1,4.5878,1018.7,83.675,3.5758,23.979,1086.2,549.83,134.67,11.898,0.32663,81.952
1,2,4.2932,1018.3,84.235,3.5709,23.951,1086.1,550.05,134.67,11.892,0.44784,82.377
2,3,3.9045,1018.4,84.858,3.5828,23.990,1086.5,550.19,135.10,12.042,0.45144,83.776
3,4,3.7436,1018.3,85.434,3.5808,23.911,1086.5,550.17,135.03,11.990,0.23107,82.505
4,5,3.7516,1017.8,85.182,3.5781,23.917,1085.9,550.00,134.67,11.910,0.26747,82.028


In [8]:
def creer_modeles_de_base():
    # Liste des modèles de base
    modeles = [ExtraTreesRegressor(), RandomForestRegressor(), DecisionTreeRegressor(), KNeighborsRegressor()]
    return modeles


def split_dataset(X, y, test_size=0.2, random_state=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, 
                                                        shuffle=True,random_state=random_state)
    return X_train, X_test, y_train, y_test


def evaluate_regressors(X, y, regressors, threshold=0.30, random_state=402):
    results = []
    
    X_train, X_test, y_train, y_test = split_dataset(X, y, threshold,random_state)
    
    for model in regressors:
        start = time()
        model.fit(X_train, y_train)
        train_time = time() - start
        start = time()
        y_pred_arr = model.predict(X_test)
        y_pred = [round(x, 0) for x in y_pred_arr]
        predict_time = time() - start 
        explained_variance = explained_variance_score(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            'Model': str(model),
            'R2 Score': r2,
            #'Training Time': train_time,
            #'Prediction Time': predict_time,
            'Expl. Variance': explained_variance
        })
    
    # Créez un DataFrame à partir des résultats
    results_df = pd.DataFrame(results)
    
    # Affichez les résultats sous forme de tableau
    display(results_df)

In [9]:
target = 'NOX'
all_features = ['AT', 'AP', 'AH', 'AFDP', 'GTEP', 'TIT', 'TAT', 'TEY', 'CDP', 'CO']
best_features = ['AT', 'AP', 'AH', 'AFDP', 'TIT', 'TAT', 'CO']

In [10]:
def charger_diviser_enregistrer_dataset(df, save_path, test_size=0.2, random_state=42):
    """
    recois un dataset, le divise en ensembles d'entraînement et de test, affiche les dimensions,
    puis enregistre les ensembles dans des fichiers CSV.

    Args:
    dataset_path (str): Le chemin du fichier CSV contenant le dataset.
    save_path (str): Le chemin du répertoire où les fichiers CSV seront enregistrés.
    test_size (float, optional): La proportion d'échantillons à inclure dans l'ensemble de test.
    random_state (int, optional): La graine utilisée par le générateur de nombres pseudo-aléatoires.

    Returns:
    DataFrame: L'ensemble d'entraînement.
    DataFrame: L'ensemble de test.
    """

    # Diviser le dataset en train et test
    train_df, test_df = train_test_split(df, test_size=test_size, random_state=random_state, shuffle=True)

    # Afficher les dimensions de chaque ensemble de données
    print("Dimensions de data_train : ", train_df.shape)
    print("Dimensions de data_test : ", test_df.shape)

    # Enregistrer les datasets dans des fichiers CSV
    train_df.to_csv(save_path + "data_train.csv", index=False)
    test_df.to_csv(save_path + "data_test.csv", index=False)

    return train_df, test_df 


save_path = "/Users/surelmanda/Downloads/AirGUARD/Gas_Turbine_Streaming/data/"

train_data, test_data = charger_diviser_enregistrer_dataset(data, save_path)


Dimensions de data_train :  (29386, 12)
Dimensions de data_test :  (7347, 12)


In [11]:
# Liste des modèles de base
regressors = creer_modeles_de_base()

# Utilisation de la fonction
X, y, threshold = train_data[best_features], train_data[target], 0.30

evaluate_regressors(X, y, regressors)

,Model,R2 Score,Expl. Variance
0,ExtraTreesRegressor(),0.869840,0.869958
1,RandomForestRegressor(),0.850040,0.850155
2,DecisionTreeRegressor(),0.702368,0.702619
3,KNeighborsRegressor(),0.751055,0.752807


In [12]:
# Liste des modèles de base
regressors = creer_modeles_de_base()

# Utilisation de la fonction
X, y, threshold = train_data[all_features], train_data[target], 0.30

evaluate_regressors(X, y, regressors)

,Model,R2 Score,Expl. Variance
0,ExtraTreesRegressor(),0.895530,0.895605
1,RandomForestRegressor(),0.872550,0.872662
2,DecisionTreeRegressor(),0.739992,0.740213
3,KNeighborsRegressor(),0.798315,0.799898


In [13]:
def entrainer_modele(model, dataset, features, target):
    """
    Fonction pour entraîner le modèle et renvoyer le modèle entraîné.

    Parameters:
    - model: Le modèle de machine learning à entraîner.
    - dataset: Le jeu de données complet.
    - features: Les colonnes utilisées comme caractéristiques.
    - target: La colonne utilisée comme cible.

    Returns:
    - model_entraine: Le modèle entraîné.
    """

    # Séparation des données en X (caractéristiques) et y (cible)
    X = dataset[features]
    y = dataset[target]

    # Entraînement du modèle sur l'ensemble d'entraînement
    model.fit(X, y)

    # Retourne le modèle entraîné
    return model


In [14]:
import os
import joblib
import sklearn

def sauvegarder_modele(model, dossier, nom_fichier):
    """
    Sauvegarde un modèle dans un dossier spécifique.

    Args:
    model: Le modèle que vous souhaitez sauvegarder.
    dossier (str): Le nom du dossier où le modèle sera sauvegardé.
    nom_fichier (str): Le nom du fichier de sauvegarde.
    """
    chemin_complet = f"{dossier}/{nom_fichier}"

    # Créer le dossier s'il n'existe pas encore
    try:
        os.makedirs(dossier)
    except FileExistsError:
        pass

    # Ajouter la version de Scikit-learn comme métadonnée
    model.sklearn_version = sklearn.__version__

    # Sauvegarder le modèle dans le dossier spécifié
    joblib.dump(model, chemin_complet)
    print(f"Le modèle a été sauvegardé avec succès dans '{chemin_complet}'.")


In [15]:
# Création du modèle ExtraTreesRegressor
mon_modele = ExtraTreesRegressor()

# Entraînement du modèle avec la fonction personnalisée
print("Processus d'entraînement du modèle...")
mon_modele_entraine = entrainer_modele(mon_modele, train_data, all_features, target)
print("Processus d'entraînement terminé.")

# Sauvegarde du modèle entraîné
print("Sauvegarde du modèle entraîné...")
modele_entrene = mon_modele_entraine # Remplacez cela par votre propre modèle entraîné
dossier_sauvegarde = "Models"
nom_fichier_sauvegarde = "modele_entrene.joblib"
sauvegarder_modele(modele_entrene, dossier_sauvegarde, nom_fichier_sauvegarde)
print("Sauvegarde du modèle terminée.")

Processus d'entraînement du modèle...
Processus d'entraînement terminé.
Sauvegarde du modèle entraîné...
Le modèle a été sauvegardé avec succès dans 'Models/modele_entrene.joblib'.
Sauvegarde du modèle terminée.
